In [1]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import pyarrow.feather as ft
import matplotlib.pyplot as plt

from tqdm import tqdm

%matplotlib inline
warnings.simplefilter('ignore')

In [2]:
PATH = '../data'

TRAIN_DATA = 'project_train.f'
TEST_DATA = 'project_test_public.f'
OPTION = 'option_names.csv'

TRAIN_PATH = os.path.join(PATH, TRAIN_DATA)
TEST_PATH = os.path.join(PATH, TEST_DATA)
OPTION_PATH = os.path.join(PATH, OPTION)

train_df = ft.read_feather(TRAIN_PATH)
test_df = ft.read_feather(TEST_PATH)
options = pd.read_csv(OPTION_PATH)

In [3]:
train_df.shape, test_df.shape

((1424484, 70), (9780, 66))

In [4]:
pd.set_option('display.max_columns', None)
train_df.head(10)

,actual_price,price,start_date,close_date,sale_end_date,brand,model,generation,modification,equipment,body_type,drive_type,transmission_type,engine_type,doors_number,color,pts,year,mileage,owners_count,steering_wheel,latitude,longitude,crashes,is_taxi,is_pledged,is_restrictions,is_carsharing,description,Передняя левая фара,Передняя правая фара,Правое зеркало,Левое зеркало,Капот,Заднее правое крыло,Заднее левое крыло,Передняя левая дверь,Крыша,Переднее правое крыло,Заднее стекло,Задний бампер,Передняя правая дверь,Задняя правая фара,Дверь багажника,Переднее левое крыло,Лобовое стекло,Задняя правая дверь,Передний бампер,Задняя левая фара,Задняя левая дверь,audiosistema,diski,electropodemniki,fary,salon,upravlenie_klimatom,usilitel_rul,aktivnaya_bezopasnost_mult,audiosistema_mult,shini_i_diski_mult,electroprivod_mult,fary_mult,multimedia_navigacia_mult,obogrev_mult,pamyat_nastroek_mult,podushki_bezopasnosti_mult,pomosh_pri_vozhdenii_mult,protivoygonnaya_sistema_mult,salon_mult,upravlenie_klimatom_mult
0,NaN,1950000.000000000000000,2023-02-18 13:37:49.603445,2023-02-19 15:06:28.603039,NaT,Volkswagen,Passat,B8 (2014—2020),2.0 TDI DSG (150 л.с.),Highline,Универсал,Передний,Робот,Дизель,5,Красный,Электронный,2017,122000,2,Левый,52.608782,39.599346,NaN,NaN,NaN,NaN,NaN,Продаю свой личный автомобиль VOLKSWAGEN PASSA...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,13777.0,13796.0,13728.0,13780.0,13716.0,13711.0,13707.0,"[13756,13757,13758,13759,13760,13761,13762]",None,[13810],[13729],"[13783,13784]","[13763,13764,13765,13766,13767,13768,13769,137...","[13721,13722,13723,13724]",None,"[13751,13752,13753,13754,13755]","[13739,13740,13741,13742,13744,13745,13746]","[13747,13748,13749]",[13719],[13713]
1,NaN,1999000.000000000000000,2023-02-11 19:49:43.158681,2023-02-24 11:04:43.685727,NaT,Volkswagen,Passat,B8 (2014—2020),2.0 TDI DSG (150 л.с.),Highline,Седан,Передний,Робот,Дизель,4,Чёрный,None,2018,74000,2,Левый,64.539393,40.516939,NaN,NaN,NaN,NaN,NaN,Автомобиль приобретался у официального дилера ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,13777.0,13797.0,13728.0,13782.0,13715.0,13711.0,13707.0,"[13756,13757,13758,13759,13760,13761,13762]",None,[13810],"[13729,13733]","[13783,13784,13785]","[13763,13764,13765,13767,13768,13769,13770,137...","[13721,13723,13724]",None,"[13751,13753,13754,13755]","[13739,13740,13741,13742,13744,13745,13746]","[13748,13749]",[13719],"[13712,13713]"
2,NaN,1780000.000000000000000,2023-02-04 14:39:08.813905,2023-02-24 20:48:25.210683,NaT,Volkswagen,Passat,B8 (2014—2020),2.0 TDI DSG (150 л.с.),None,Седан,Передний,Робот,Дизель,4,Чёрный,Оригинал,2018,36000,1,Левый,55.755814,37.617635,NaN,NaN,NaN,NaN,NaN,Без пробега по РФ.Как новая.Идеальный вариант ...,None,None,None,None,None,None,None,None,None,Скол или царапина,None,None,None,None,None,None,None,None,None,None,None,NaN,13796.0,13728.0,13781.0,13717.0,13711.0,13707.0,"[13756,13757,13758]",None,None,"[13729,13731,13733]",[13783],"[13768,13772]",None,None,"[13751,13752,13753,13754]","[13740,13741,13742,13744,13745,13746]",[13749],None,None
3,NaN,1900000.000000000000000,2023-02-04 08:04:20.229815,2023-02-15 17:20:49.248078,NaT,Volkswagen,Passat,B8 (2014—2020),2.0 TDI DSG (150 л.с.),Highline,Седан,Передний,Робот,Дизель,4,Серый,Электронный,2018,180000,1,Левый,55.557174,37.708644,NaN,NaN,NaN,NaN,NaN,"Продам надежный ухоженный автомобиль, я собств...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,13776.0,13797.0,13728.0,NaN,13715.0,13711.0,13707.0,"[13756,13757,13758,13759,13760,13762]",None,[13810],"[13729,13731,13733]","[13783,13784,13785]","[13763,13764,13765,13768,13769,13770,13771,13772]","[13721,13723,13724]",None,"[13751,13752,13753,13754,13755]","[13739,13740,13741,13742,13743,13744,13745,13746]","[13747,13749]",[13719],"[13712,13713]"
4,NaN,1650000.000000000000000,2023-02-04 01:42:

In [5]:
train_df.nunique()

actual_price                      13964
price                             32423
start_date                      1343058
close_date                      1343056
sale_end_date                       240
                                 ...   
podushki_bezopasnosti_mult          278
pomosh_pri_vozhdenii_mult          4538
protivoygonnaya_sistema_mult         64
salon_mult                            4
upravlenie_klimatom_mult              4
Length: 70, dtype: int64

In [6]:
train_df.actual_price.notnull().sum() / train_df.shape[0], train_df.actual_price.isnull().sum() / train_df.shape[0]

(0.05716175120254071, 0.9428382487974593)

In [7]:
options.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
id,13705,13706,13707,13709,13710,13711,13712,13713,13715,13716,13717,13718,13719,13720,13721,13722,13723,13724,13725,13727,13728,13729,13730,13731,13732,13733,13734,13735,13736,13737,13738,13739,13740,13741,13742,13743,13744,13745,13746,13747,13748,13749,13750,13751,13752,13753,13754,13755,13756,13757,13758,13759,13760,13761,13762,13763,13764,13765,13766,13767,13768,13769,13770,13771,13772,13774,13775,13776,13777,13778,13780,13781,13782,13783,13784,13785,13786,13787,13788,13789,13790,13791,13792,13793,13794,13795,13796,13797,13798,13799,13800,13801,13802,13803,13804,13805,13806,13807,13808,13809,13810
viewItemLabel,Гидравлический,Электрогидравлический,Электрический,Кондиционер,Климат-контроль однозонный,Климат-контроль многозонный,Управление на руле,Атермальное остекление,Кожа,Ткань,Велюр,Комбинированный,Кожаный руль,Люк,Передних сидений,Задних сидений,Зеркал,Заднего стекла,Руля,Только передние,Передние и задние,Передних сидений,Задних сидений,Зеркал,Рулевой колонки,Складывания зеркал,Передних сидений,Задних сидений,Зеркал,Рулевой колонки,Автоматический парковщик,Датчик дождя,Датчик света,Парктроник задний,Парктроник передний,Система контроля слепых зон,Камера заднего вида,Круиз-контроль,Бортовой компьютер,Сигнализация,Центральный замок,Иммобилайзер,Спутник,Фронтальные,Коленные,Шторки,Боковые передние,Боковые задние,Антиблокировка тормозов,Антипробуксовка,Курсовая устойчивость,Распред. тормозных усилий,Экстренное торможение,Блок. дифференциала,Обнаружение пешеходов,CD/DVD/Blu-ray,MP3,Радио,TV,Видео,Управление на руле,USB,AUX,Bluetooth,GPS-навигатор,2 колонки,4 колонки,6 колонок,8+ колонок,Сабвуфер,Галогенные,Ксеноновые,Светодиодные,Противотуманные,Омыватели фар,Адаптивное освещение,"7""","8""","9""","10""","11""","12""","13""","14""","15""","16""","17""","18""","19""","20""","21""","22""","23""","24""","25""","26""","27""","28""","29""","30""",Зимние шины в комплекте
